In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.metrics import f1_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob
print(os.listdir("../input"))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
gc.collect()

In [ ]:

from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
data_folder = Path("/kaggle/input/emergency")

In [ ]:
data_folder

In [ ]:
data_path = "/kaggle/input/emergency/train_SOaYf6m/images"
path = os.path.join(data_path , "*jpg")

In [ ]:
files = glob.glob(path)

data=[]
for file in files:
    image = cv2.imread(file)
    data.append(image)

In [ ]:
train_df = pd.read_csv('/kaggle/input/emergency/train_SOaYf6m/train.csv')
test_df = pd.read_csv('/kaggle/input/emergency/test_vc2kHdQ.csv')
submit = pd.read_csv('/kaggle/input/emergency/sample_submission_yxjOnvz.csv')

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.groupby('emergency_or_not').count()

In [ ]:
sns.countplot(x='emergency_or_not' , data=train_df)

In [ ]:
train_images = data[:1646]
test_images= data[1646:]


In [ ]:
#tfms = get_transforms(do_flip=True,flip_vert=False ,max_rotate=17.2, max_zoom=1.22, max_lighting=0.42, max_warp=0.0, p_affine=0.75,
#                      p_lighting=0.75)
#, xtra_tfms=zoom_crop(scale=(0.9,1.8), do_rand=True, p=0.8))

tfms = get_transforms(do_flip=False,flip_vert=False ,max_rotate=11.0, max_zoom=1.52, max_lighting=0.28, max_warp=0.0, p_affine=0.75,
                      p_lighting=0.75)

## create databunch of test set to be passed
test_img = ImageList.from_df(test_df, path=data_folder/'train_SOaYf6m', folder='images')

In [ ]:
np.random.seed(145)
## create source of train image databunch
src = (ImageList.from_df(train_df, path=data_folder/'train_SOaYf6m', folder='images')
       .split_by_rand_pct(0.2)
       #.split_none()
       .label_from_df()
       .add_test(test_img))

In [ ]:
data = (src.transform(tfms, size=299,padding_mode='reflection',resize_method=ResizeMethod.SQUISH)
        .databunch(path='.', bs=32, device= torch.device('cuda:0')).normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,12))


In [ ]:

print(data.classes)

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
from fastai import *
from fastai.vision import *
from efficientnet_pytorch import EfficientNet


def get_model(pretrained=True,**kwargs):
    model = EfficientNet.from_pretrained('efficientnet-b3')
    model._fc = nn.Linear(model._fc.in_features,data.c)
    return model

In [ ]:
#learn = cnn_learner(data=data, base_arch=models.resnet101, metrics=[FBeta(beta=1, average='macro'), accuracy],
#                    callback_fns=ShowGraph)


#learn = cnn_learner(data=data, base_arch=models.densenet161, metrics=[FBeta(beta=1, average='macro'), accuracy],
#                     callback_fns=ShowGraph).mixup()
learn = Learner(data, get_model(), metrics=[FBeta(beta=1, average='macro'), accuracy],
                     callback_fns=ShowGraph)

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 3e-03
#learn.fit_one_cycle(10, slice(lr))
learn.fit_one_cycle(15, slice(lr), wd=0.2)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)


In [ ]:

# train for  more cycles after unfreezing
learn.fit_one_cycle(10,slice(1e-04,lr/8),wd=0.15)
#learn.fit_one_cycle(10, slice(5e-06, lr/8))

In [ ]:
learn.freeze_to(-3)

In [ ]:
## finding the LR
learn.lr_find()
learn.recorder.plot(suggestion=True)


In [ ]:
learn.fit_one_cycle(6, slice(1e-06, lr/10),wd=0.1)

In [ ]:
learn.freeze_to(-2)

In [ ]:
learn.fit_one_cycle(6, slice(5e-07, lr/20),wd=0.1)

In [ ]:
learn.freeze_to(-1)

In [ ]:
learn.fit_one_cycle(5, slice(1e-07, lr/30),wd=0.05)

In [ ]:
learn.fit_one_cycle(6, slice(1e-07, lr/100))

In [ ]:

#lets see the most mis-classified images (on validation set)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(7,6))

In [ ]:

interp.plot_confusion_matrix(figsize=(6,6), dpi=60)


In [ ]:
interp.most_confused(min_val=4) ## on validation set


In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)
#preds,_ = learn.get_preds(ds_type = DatasetType.Test)
labelled_preds = [np.argmax(preds[i])+1 for i in range(len(preds))]

labelled_preds = np.array(labelled_preds)

In [ ]:
labelled_preds

In [ ]:
temp = test_df.copy()

In [ ]:
temp ['emergency_or_not'] = labelled_preds

In [ ]:
temp.head()

In [ ]:
to_submit = temp[['image_names', 'emergency_or_not']]

In [ ]:
to_submit.emergency_or_not.value_counts()

In [ ]:
to_submit.head()

In [ ]:
to_submit.to_csv('efficientnetb3.csv',index = False)